# Exploratory Data Analysis: _Checking CWS to ZCTA Mapping_

In this file, we'll check the CWS to ZCTA crosswalks created by `01-data-cleaning-mapping-CWS-ZCTA.py` to confirm data availability and correspondence across ZCTA 2000 and 2010 definitions. Then, we'll generate some basic maps showing the overlay output.

In [ ]:
# Loading required libraries and setting working directory

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# -----------------------------------------------------------------------------
# Inspecting Crosswalk Files Created Above
# -----------------------------------------------------------------------------

# Load both datasets created above
crosswalk_2000 = pd.read_csv(os.path.join(
    config.PROCESSED_DATA_DIR, 
    "CA-PWS-to-ZTCA-2000-crosswalk.csv"
))

crosswalk_SF_2000 = gpd.read_file(os.path.join(
    config.PROCESSED_DATA_DIR,
    "CA-PWS-to-ZTCA-2000-crosswalk-SF.geojson"
))

# Check total overlap across each ZTCA for each CWS
total_overlap = crosswalk_2000.groupby("SABL_PWSID")["coverage_frac_ztca"].sum().reset_index()
total_overlap = total_overlap.sort_values("coverage_frac_ztca", ascending=False)
print("\nTop 10 PWS systems by total ZTCA overlap:")
print(total_overlap.head(10))

# Define list of all zip codes included in the intersections output
zcta_list = crosswalk_SF_2000["ZCTA5CE00"].unique()

# Load ZTCA shapes for visualization (subsetted to ones we need)
ztca_for_plot = ztca_equal_area[ztca_equal_area["ZCTA5CE00"].isin(zcta_list)]

# Plot ZTCA and PWS boundaries
fig, ax = plt.subplots(figsize=(12, 10))

# Base layer - ZTCA boundaries in light blue with transparent fill
ztca_for_plot.plot(ax=ax, color="skyblue", edgecolor="skyblue", alpha=0.2, label="ZTCA Boundaries")

# Middle layer - CWS boundaries in red with no fill
# Only plotting first 10 for visibility
pws_subset = pws_with_violations[pws_with_violations["SABL_PWSID"].isin(PWS_with_violation_IDs[:10])]
pws_subset.plot(ax=ax, color="red", facecolor="none", linewidth=1, label="CWS Boundaries")

# Add labels and title
ax.set_title("Water System and ZTCA Boundaries with Intersections", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()

# Save figure
plt.savefig(os.path.join(
    config.OUTPUT_FIG_DIR,
    "water-system-ztca-boundaries.png"
), dpi=300, bbox_inches="tight")

print("Analysis complete. Visualization saved.")